In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !unzip /content/drive/MyDrive/ImCap/mlproj.zip

In [3]:
import json
data = json.load(open("mlproj/data/memes-trainval.json", "rb"))

In [4]:
import pandas as pd

In [5]:
df = pd.DataFrame(data)

In [6]:
df.columns

Index(['category', 'img_captions', 'meme_captions', 'title', 'url',
       'img_fname', 'metaphors', 'post_id'],
      dtype='object')

In [7]:
D = df[["img_captions", "title", "img_fname", "metaphors"]]
Y = df[["meme_captions"]]

In [8]:
D

,img_captions,title,img_fname,metaphors
0,[Person in Spider Man outfit gives a lecture o...,For real though,memes_bpet7l.png,"[{'metaphor': 'Spider Man outfit', 'meaning': ..."
1,"[Two dogs carry a white flag, Identical chubby...",And that's a fact,memes_ctxvmu.png,"[{'metaphor': 'Two dogs', 'meaning': 'Meme pos..."
2,"[man is very dissatisfied with what he sees, Q...",It was horrible,memes_cxvuqd.png,"[{'metaphor': 'man', 'meaning': 'Meme poster'}..."
3,[A man that is crying and a man that is laughi...,This is why Reddit is better,memes_cyop7a.png,"[{'metaphor': 'A man', 'meaning': 'Meme poster..."
4,[a cop is chasing a thief with a shopping cart...,The Area 51 raid is still happening right?,memes_cz2i20.png,"[{'metaphor': 'a cop', 'meaning': 'Meme poster..."
...,...,...,...,...
5818,[The middle aged somewhat bolding man is at a ...,Posting memes until I get my master's degree d...,memes_zd0n76.png,"[{'metaphor': 'somewhat bolding man', 'meaning..."
5819,[A woman is deep in thought throughout the day.],*pretends to think*,memes_zd0pqv.png,"[{'metaphor': 'A woman', 'meaning': 'the poste..."
5820,[Captain America talks to a man in a elevator ...,overreaction indeed,memes_zd0rgl.png,"[{'metaphor': 'Captain America', 'meaning': 'M..."
5821,[two drawn faces face to face and right one is...,Anime watchers be like,memes_zd0s2c.png,"[{'metaphor': 'two drawn faces', 'meaning': 'a..."


In [9]:
# !pip install git+https://github.com/huggingface/transformers.git@main

In [10]:
D.loc[4, :]

img_captions    [a cop is chasing a thief with a shopping cart...
title                  The Area 51 raid is still happening right?
img_fname                                        memes_cz2i20.png
metaphors       [{'metaphor': 'a cop', 'meaning': 'Meme poster...
Name: 4, dtype: object

In [11]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset.loc[idx, :]
        data = ", ".join(item.meme_captions)
        data = data.strip("Meme poster is trying to convey that ").strip("Meme poster ").strip("The meme poster ").strip()
        encoding = self.processor(images=Image.open(f"mlproj/data/memes/{item.img_fname}"), text=data, padding="max_length", return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding


In [12]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
# model.load_state_dict(torch.load('/kaggle/input/modelpretrined/model_weights (1).pth'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
train_dataset = ImageCaptioningDataset(df, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)

In [14]:
import torch
torch.cuda.is_available()

True

In [15]:
torch.cuda.empty_cache()

In [24]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
count = 0
model.train()

for epoch in range(1):
  print("Epoch:", epoch)
  torch.save(model.state_dict(), 'model.pth')
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss

    print("Loss:", loss.item())
    if idx/100 == 0:
        torch.save(model.state_dict(), 'model_weights.pth')
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    if loss < 0.1:
        break


Epoch: 0
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
Loss: 13.14870834350586
Loss: 10.32192325592041
Loss: 10.205180168151855
Loss: 10.239234924316406
Loss: 10.20363998413086
Loss: 10.189112663269043
Loss: 10.216890335083008
Loss: 10.165626525878906
Loss: 10.123507499694824
Loss: 10.180231094360352
Loss: 10.131760597229004
Loss: 10.100406646728516
Loss: 9.854722023010254
Loss: 9.297161102294922
Loss: 8.946335792541504
Loss: 8.747649192810059
Loss: 8.507657051086426
Loss: 8.33669376373291
Loss: 8.091744422912598
Loss: 7.967598915100098
Loss: 7.741539001464844
Loss: 7.593621253967285
Loss: 7.409764289855957
Loss: 7.270691871643066
Loss: 7.12330436706543
Loss: 6.97792387008667
Loss: 6.803122043609619
Loss: 6.708864212036133
Loss: 6.543376445770264
Loss: 6.384299278259277
Loss: 6.233576774597168
Loss: 6.09076452255249
Loss: 5.96860027

In [21]:
test_data = json.load(open("mlproj/data/mlr_captioning_TEST.json", "rb"))
test_df = pd.DataFrame(test_data)
test_df = test_df[["post_id", "img_fname"]]

outputs = []

for ix, img_fname in enumerate(test_df.img_fname.values):
    try:
        inputs = processor(images=np.array(Image.open(f"mlproj/data/memes/{img_fname}")), return_tensors="pt").to(device)
        pixel_values = inputs.pixel_values
        generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        outputs.append(generated_caption)
    except ValueError:
        outputs.append("None")
        continue

    if ix % 10 == 0:
        print(ix, end = " ")

test_df.loc[:, "meme_captions"] = outputs
test_df.to_csv("submission.csv", index=False)

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 270 280 290 300 310 330 340 360 370 380 390 400 410 420 430 450 460 470 480 500 510 520 530 540 550 

In [22]:
test_df = test_df[["post_id", "meme_captions"]]
test_df.to_csv("submission.csv", index=False)